In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

In [ ]:
def loadcsv(csv_path, sep=",", **kwargs):
  dfTreeInv = pd.read_csv(csv_path, sep=sep)

  dfTreeInv[dfTreeInv.select_dtypes(['object']).columns] = dfTreeInv.select_dtypes(['object']).apply(lambda x: x.str.strip())
  dfTreeInv = dfTreeInv.drop(columns=['GeometryEasting', 'GeometryNorthing'])   # drop geo east, north
  dfTreeInv['SPECIES_NAME']= dfTreeInv['SPECIES_NAME'].str.title()    #title species name
  dfTreeInv.loc[dfTreeInv['ROAD_NAME'].isnull(), 'ROAD_NAME'] = 'UnNamed'  # fill empty road name by 'UnNamed'

  dfTemp=dfTreeInv.drop(columns=['OVT_ID', 'LU_HMD', 'ROAD_NAME'])
  dfTemp.columns.to_list()

  # Filter out rows with NaN, None, or empty strings
  missing_dfTemp = dfTemp[dfTemp.isna().any(axis=1) | (dfTemp == '').any(axis=1)]
  num_missing = set(missing_dfTemp['OBJECTID'])
  missingRows = ', '.join(map(str, num_missing))

  dfTreeImport = dfTreeInv[~dfTreeInv['OBJECTID'].isin(missing_dfTemp['OBJECTID'])].copy()
  importedRows = dfTreeImport.shape[0]

  # gdf
  dfTreeImport['geometry'] = dfTreeImport.apply(lambda row: Point(row['EASTING'], row['NORTHING']), axis=1) 
  gdfTreeImport = gpd.GeoDataFrame(dfTreeImport, geometry='geometry', crs=2326).to_crs(4326) # change to crs4326 fm 2326
  gdfTreeImport.columns =gdfTreeImport.columns.str.lower()

  return gdfTreeImport, importedRows, missing_dfTemp, missingRows, num_missing

In [ ]:
gdfTreeImport, importedRows, missing_dfTemp, missingRows = loadcsv('./externalSources/VIS_INV_TREE_CSDI_202512160000_converted.csv',sep=",")